# 🛍️ ShopSense — Phase 2: Semantic Retrieval with FAISS
**Environment:** VS Code (local machine)

### What this phase does:
1. Loads the 10,000 clean products from Phase 1
2. Embeds every product description using `all-MiniLM-L6-v2` (a fast, lightweight sentence transformer)
3. Stores all embeddings in a FAISS index for fast similarity search
4. Builds a `search()` function that takes a natural language query and returns the top 20 matching products

### New files created:
```
shopsense/
├── data/
│   ├── products_clean.csv       ← from Phase 1
│   ├── faiss_index.bin          ← FAISS index (new)
│   └── faiss_metadata.json      ← maps index positions → product info (new)
├── data_preparation.ipynb
└── semantic_retrieval.ipynb     ← this file
```

## Step 0 — Install Dependencies

In [ ]:
import sys
!{sys.executable} -m pip install sentence-transformers faiss-cpu pandas numpy tqdm -q
print('✅ Dependencies installed.')

## Step 1 — Load Products from Phase 1

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('data')

df = pd.read_csv(DATA_DIR / 'products_clean.csv')

# Drop rows with missing descriptions (safety check)
df = df.dropna(subset=['description']).reset_index(drop=True)

print(f'✅ Loaded {len(df):,} products')
print(f'Columns: {df.columns.tolist()}')
df.head(3)

## Step 2 — Prepare Text for Embedding
We combine `title` + `description` into one string per product.
Title is repeated at the front because it carries the most signal.

In [ ]:
def build_embed_text(row):
    """Combine title + description into one rich string for embedding."""
    title = str(row.get('title', '') or '').strip()
    desc  = str(row.get('description', '') or '').strip()
    # Truncate description to 300 chars to avoid token limit issues
    desc  = desc[:300]
    return f"{title}. {desc}"

df['embed_text'] = df.apply(build_embed_text, axis=1)

print(f'✅ Embed text prepared for {len(df):,} products')
print(f'Avg embed text length: {df["embed_text"].str.len().mean():.0f} chars')
print(f'\nSample embed text:')
print(df['embed_text'].iloc[0])

## Step 3 — Load Sentence Transformer Model
`all-MiniLM-L6-v2` is ~80MB, downloads once and caches locally.
It produces 384-dimensional embeddings and is fast on CPU.

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = 'all-MiniLM-L6-v2'

print(f'Loading model: {MODEL_NAME}')
print('First run downloads ~80MB — cached after that...')

model = SentenceTransformer(MODEL_NAME)

print(f'✅ Model loaded')
print(f'   Embedding dimension : {model.get_sentence_embedding_dimension()}')
print(f'   Max sequence length : {model.max_seq_length}')

## Step 4 — Generate Embeddings for All Products
Embeds all 10,000 product descriptions.
Takes ~5-10 minutes on CPU — grab a coffee ☕

In [ ]:
import numpy as np
import time

texts = df['embed_text'].tolist()

print(f'Embedding {len(texts):,} product descriptions...')
print('This takes ~5-10 min on CPU. Progress bar below:')

start = time.time()

embeddings = model.encode(
    texts,
    batch_size=64,          # process 64 at a time
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # L2 normalize — makes cosine similarity = dot product
)

elapsed = time.time() - start

print(f'\n✅ Embeddings generated')
print(f'   Shape     : {embeddings.shape}  (products × embedding_dim)')
print(f'   Data type : {embeddings.dtype}')
print(f'   Time taken: {elapsed/60:.1f} min')

## Step 5 — Build FAISS Index
FAISS (Facebook AI Similarity Search) stores all embeddings and lets us find the nearest neighbors to any query vector in milliseconds.

We use `IndexFlatIP` — Inner Product index. Since embeddings are L2 normalized, inner product = cosine similarity.

In [ ]:
import faiss

EMBEDDING_DIM = embeddings.shape[1]  # 384 for MiniLM

# Build the index
index = faiss.IndexFlatIP(EMBEDDING_DIM)   # IP = Inner Product (cosine sim)
index.add(embeddings.astype('float32'))    # FAISS requires float32

print(f'✅ FAISS index built')
print(f'   Index type       : IndexFlatIP (exact cosine similarity)')
print(f'   Embedding dim    : {EMBEDDING_DIM}')
print(f'   Vectors indexed  : {index.ntotal:,}')

## Step 6 — Save FAISS Index + Metadata

In [ ]:
import json

# Save FAISS index
index_path = DATA_DIR / 'faiss_index.bin'
faiss.write_index(index, str(index_path))
print(f'✅ FAISS index saved → {index_path} ({index_path.stat().st_size/1024:.0f} KB)')

# Save metadata — maps each FAISS position (0, 1, 2...) to product info
# This is how we go from "index position 42" back to the actual product
metadata = df[['asin', 'title', 'price', 'rating', 'rating_count', 'category', 'description']].to_dict(orient='records')

meta_path = DATA_DIR / 'faiss_metadata.json'
with open(meta_path, 'w') as f:
    json.dump(metadata, f)

print(f'✅ Metadata saved    → {meta_path} ({meta_path.stat().st_size/1024:.0f} KB)')
print(f'\n   Metadata contains {len(metadata):,} product records')
print(f'   Fields: {list(metadata[0].keys())}')

## Step 7 — Build the Search Function
This is the core of Phase 2. Takes a natural language query, embeds it, and returns the top-K most similar products.

In [ ]:
def search(query: str, top_k: int = 20) -> list[dict]:
    """
    Search for products using natural language.

    Args:
        query  : Natural language search string e.g. 'compact coffee maker for small kitchen'
        top_k  : Number of results to return (default 20, reranker will pick top 5)

    Returns:
        List of dicts with product info + similarity score
    """
    # 1. Embed the query using the same model
    query_embedding = model.encode(
        [query],
        normalize_embeddings=True,
        convert_to_numpy=True
    ).astype('float32')

    # 2. Search the FAISS index
    scores, indices = index.search(query_embedding, top_k)

    # 3. Build results list
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:   # FAISS returns -1 for empty slots
            continue
        product = metadata[idx].copy()
        product['similarity_score'] = round(float(score), 4)
        results.append(product)

    return results


print('✅ search() function ready.')
print('Usage: results = search("your query here", top_k=20)')

## Step 8 — Test the Search Function
Let's run a few queries and see what comes back!

In [ ]:
def display_results(query: str, top_k: int = 5):
    """Pretty print search results."""
    print(f'🔍 Query: "{query}"')
    print('─' * 65)
    results = search(query, top_k=top_k)
    for i, r in enumerate(results, 1):
        price_str = f"${r['price']:.2f}" if r['price'] else 'N/A'
        print(f"{i}. {r['title'][:70]}")
        print(f"   Price: {price_str}  |  Rating: {r['rating']}⭐ ({r['rating_count']:,} reviews)")
        print(f"   Similarity: {r['similarity_score']}")
        print()
    print('=' * 65)

# ── Test Query 1 ──
display_results('compact coffee maker for small kitchen', top_k=5)

In [ ]:
# ── Test Query 2 ──
display_results('non stick frying pan under $30', top_k=5)

In [ ]:
# ── Test Query 3 ──
display_results('gift for someone who loves cooking', top_k=5)

In [ ]:
# ── Test Query 4 — try your own! ──
display_results('storage solution for small apartment', top_k=5)

## Step 9 — Reload Test (verify saved index works)
Simulates loading the index fresh — exactly what Phase 3 will do.

In [ ]:
print('Testing reload from disk...')

# Load index from disk
index_loaded = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))

# Load metadata from disk
with open(DATA_DIR / 'faiss_metadata.json') as f:
    metadata_loaded = json.load(f)

print(f'✅ Index reloaded — {index_loaded.ntotal:,} vectors')
print(f'✅ Metadata reloaded — {len(metadata_loaded):,} records')

# Quick sanity search
q_emb = model.encode(['kitchen knife set'], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
scores, indices = index_loaded.search(q_emb, 3)
print(f'\nSanity search → "kitchen knife set":')
for score, idx in zip(scores[0], indices[0]):
    print(f'  [{score:.4f}] {metadata_loaded[idx]["title"][:70]}')

## Step 10 — Final Summary

In [ ]:
print('=' * 55)
print('     PHASE 2 COMPLETE — SUMMARY')
print('=' * 55)
print(f'  Model              : {MODEL_NAME}')
print(f'  Embedding dim      : {EMBEDDING_DIM}')
print(f'  Products indexed   : {index.ntotal:,}')
print(f'  Index type         : IndexFlatIP (cosine similarity)')
print()
print('  Files created:')
for fname in ['faiss_index.bin', 'faiss_metadata.json']:
    fpath = DATA_DIR / fname
    print(f'    ✅ data/{fname} ({fpath.stat().st_size/1024:.0f} KB)')
print()
print('  ➡️  Next: Phase 3 — Cross-Encoder Reranking (Colab, GPU)')
print('=' * 55)

## Push to GitHub
```bash
git add semantic_retrieval.ipynb
git commit -m "phase 2: semantic retrieval with FAISS complete"
git push
```
> `data/` is still in `.gitignore` so the large `.bin` and `.json` files won't be pushed.